In [2]:
pip install beautifulsoup4 requests pandas lxml fake_useragent

Note: you may need to restart the kernel to use updated packages.


In [1]:
import requests # sends http requests
from bs4 import BeautifulSoup # used  for parsing html data
import pandas as pd # handles data frames and data manipulation
import time # used for sleep functions to delay scraping requests
import random # generates random delays between requests
import re # used for pattern matching
import csv # writes data to .csv file

# AMAZON URL FOR DATA ENGINEERING BOOKS SEARCH RESULTS
URL = "https://www.amazon.com/s?k=data+engineering+books"

# USER-AGENT LIST TO AVOID DETECTION BY AMAZON
USER_AGENTS = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.0.0 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.83 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36"
]

# HEADERS TO MIMIC A REAL BROWSER REQUEST FOR SCRAPING
HEADERS = {
    "User-Agent": random.choice(USER_AGENTS),  # randomly choose a user-agent to avoid detection
    "Accept-Language": "en-US,en;q=0.5"  # define acceptable languages in the request
}

# FUNCTION TO GET PUBLICATION DATE FROM BOOK'S PRODUCT PAGE
def get_publication_date_from_product_page(book_url):
    """Fetch the publication date from a book's product page."""
    try:
        # SEND REQUEST TO BOOK'S PRODUCT PAGE
        product_response = requests.get(book_url, headers=HEADERS)
        
        # CHECK IF THE PAGE LOADS SUCCESSFULLY
        if product_response.status_code == 200:
            product_soup = BeautifulSoup(product_response.text, "html.parser")  # parses html response
            # POSSIBLE LOCATIONS TO LOOK FOR PUBLICATION DATE
            possible_locations = [
                "#detailBullets_feature_div", 
                "#productDetailsTable", 
                "#prodDetails"
            ]
            for location in possible_locations:  # checks all locations
                details = product_soup.select_one(location)  # gets the element containing details
                if details:
                    detail_text = details.get_text(strip=True)  # cleans up the text
                    # SEARCH FOR A FULL DATE PATTERN
                    date_match = re.search(r"(January|February|March|April|May|June|July|August|September|October|November|December) \d{1,2}, \d{4}", detail_text)
                    if date_match:
                        return date_match.group()  # returns the found date

            # IF NO FULL DATE FOUND, SEARCH FOR A YEAR IN THE TEXT
            year_match = re.search(r"(19|20)\d{2}", product_soup.text)
            if year_match:
                return year_match.group()  # returns the year
    except Exception as e:
        print(f"Error fetching product page: {e}")  # prints error if exception error occurs
    return None  # returns none if no date is found

# FETCH MAIN SEARCH RESULTS PAGE FROM AMAZON
response = requests.get(URL, headers=HEADERS)

# CHECK IF PAGE RETRIEVAL WAS SUCCESSFUL
if response.status_code == 200:
    soup = BeautifulSoup(response.text, "html.parser")  # parses html of main page
    titles, authors, pub_dates, ratings, prices = [], [], [], [], []  # lists to store scraped data
    books = soup.find_all("div", {"data-component-type": "s-search-result"})  # finds all book entries

    # LOOP THROUGH FIRST 25 BOOKS
    for book in books[:25]:
        title_tag = book.find("h2", class_="a-size-base-plus a-spacing-none a-color-base a-text-normal")  # finds title
        title = f'"{title_tag.text.strip()}"' if title_tag else None  # encloses title in quotes

        # EXTRACT AUTHOR INFORMATION
        author_tag = book.find("div", class_="a-row a-size-base a-color-secondary")
        if author_tag:
            author_links = author_tag.find_all("a")
            author = ", ".join([a.text.strip() for a in author_links]) if author_links else None  # combines author names
        else:
            author = None

        # EXTRACT PUBLICATION DATE
        pub_date = None
        date_spans = book.find_all("span", class_="a-size-base a-color-secondary")  # finds all date spans
        for span in date_spans:
            span_text = span.get_text(strip=True)
            # SEARCH FOR A FULL DATE FORMAT
            date_match = re.search(r"(January|February|March|April|May|June|July|August|September|October|November|December) \d{1,2}, \d{4}", span_text)
            if date_match:
                pub_date = date_match.group()  # sets publication date
                break
        
        # IF NO FULL DATE, SEARCH FOR A YEAR ONLY
        if pub_date is None:
            year_match = re.search(r"(19|20)\d{2}", book.text)
            if year_match:
                pub_date = year_match.group()  # sets publication year

        # EXTRACT RATING
        rating_tag = book.find("span", class_="a-icon-alt")
        rating = rating_tag.text.strip().split()[0] if rating_tag else None  # extracts rating value

        # EXTRACT PRICE
        price_tag = book.find("span", class_="a-price-whole")
        if not price_tag:
            price_tag = book.find("span", class_="a-offscreen")
        
        if price_tag:
            price = price_tag.text.strip()
            price = f"{float(price):.2f}" if price else None  # ensures two decimal places for price
        else:
            price = None

        # EXTRACT BOOK URL FOR FURTHER DETAILS
        book_url_tag = book.find("a", class_="a-link-normal s-no-outline")
        book_url = "https://www.amazon.com" + book_url_tag["href"] if book_url_tag else None  # creates full url

        # FETCH PUBLICATION DATE FROM PRODUCT PAGE IF MISSING
        if pub_date is None and book_url:
            pub_date = get_publication_date_from_product_page(book_url)

        # APPEND EXTRACTED DATA TO LISTS
        titles.append(title)
        authors.append(author)
        pub_dates.append(pub_date)
        ratings.append(rating)
        prices.append(price)

        # RESPECT AMAZON SCRAPING GUIDELINES BY ADDING RANDOM DELAY
        time.sleep(random.uniform(1, 3))

    # CREATE PANDAS DATAFRAME WITH SCRAPED DATA
    df = pd.DataFrame({
        "Title": titles,
        "Author": authors,
        "Publication Date": pub_dates,
        "Rating": ratings,
        "Price": prices
    })

    # SAVE DATA TO CSV WITH PROPER QUOTING
    df.to_csv("amazon_data_engineering_books.csv", index=False, quoting=csv.QUOTE_ALL)

    # DISPLAY THE SCRAPED DATA
    print("\nScraped Data:\n")
    print(df.to_string(index=False))
    print("\nData successfully saved to 'amazon_data_engineering_books.csv'.")
else:
    print("Failed to retrieve webpage. Amazon may have blocked the request.")  # error handling failure in retrieval


Scraped Data:

                                                                                                                                                                                                 Title                                            Author  Publication Date Rating Price
                                                                                                                                             "Hands-On Data Engineering with R, Python and PostgreSQL"                                   Michel Ballings December 14, 2024   None 69.00
                                                                                "Data Engineering with AWS: Acquire the skills to design and build AWS-based data transformation pipelines like a pro"                                      Gareth Eagar  October 31, 2023    4.3 24.00
                                                 "Cracking the Data Engineering Interview: Land your dream job with the help of resume-building 